# 0. Setup

Script to regenerate the DetectRL data using new OpenAI models. 
In the current setup the arxiv data is processed using the  "gpt-4.1-2025-04-14" model.
The prompts of the DetectRL dataset are used. 
Has a debug mode to test the setup by generating two samples.

In [1]:
import sys
import time
import pandas as pd
import json
import nltk
from tqdm import tqdm
import ast

In [2]:
BASE_DIR = "../../"
sys.path.append(BASE_DIR)

from src.openAI_client import (
    client,
    generate_openai_response,
    generate_jsonl_file
)

from src.general_functions_and_patterns_for_detection import (
    json_path_abstract,
    RESULT_DIR
)

DEBUG = True

In [3]:
# Load JSON data
with open(json_path_abstract, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)
df.head()

,id,title,abstract,direct_prompt,llm_type,domain,prompt_few_shot,prompt_SICO,paraphrase_polish_human,paraphrase_polish_llm,...,adversarial_character_human,adversarial_character_llm,adversarial_word_human,adversarial_word_llm,adversarial_character_word_human,adversarial_character_word_llm,paraphrase_back_translation_human,paraphrase_back_translation_llm,paraphrase_dipper_human,paraphrase_dipper_llm
0,1,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbati...,This study presents a comprehensive calculatio...,ChatGPT,arxiv,"In this study, we present a comprehensive calc...","Abstract:In this study, we present a comprehen...",This study presents a comprehensive and fully ...,This study presents a comprehensive calculatio...,...,A fully differential calCulation in perturbati...,This study presents a comprehensive calculatio...,A fully disparity calculation in perturbative ...,This study presents a full calculation of prom...,A fully differential calculation in perturbati...,This study рresents a comprehensive calculatio...,The calculation of the complete difference in ...,This study lists the comprehensive calculation...,The calculation includes all next-to-leading-o...,"To determine the cross sections, we use the mo..."
1,2,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is describe...,The understanding of the dynamic evolution of ...,ChatGPT,arxiv,"In this study, we explore the evolutionary dyn...",Abstract:The evolution of the Earth-Moon syste...,The evolution of the Earth-Moon system is addr...,The dynamic evolution of the Earth-Moon system...,...,The evolution of Earth-Moon system is describe...,The understanding of the dynamic evolution of ...,The evolution of Earth-Moon system is outlines...,The understanding of the dynamic evolution of ...,The evolution of Earth-Moon system is describe...,The understanding of the dynamic changing of t...,The evolution of the global system is describe...,The understanding of the dynamic evolution of ...,The closest distance of the Moon to the Earth ...,"In this study, we present a new approach to th..."
2,3,Bosonic characters of atomic Cooper pairs acro...,We study the two-particle wave function of pai...,This article investigates the bosonic characte...,ChatGPT,arxiv,We investigate the bosonic characteristics of ...,Abstract:This article investigates the bosonic...,We investigate the characteristics of the two-...,This article delves into the examination of th...,...,We study the two-particle wave function of pai...,This article investigates the bosonic characte...,We study the two-particle wave function of pai...,This art investigates the bosonic characterist...,We study the two-particle wave function of pai...,This article investigates the bosonic characte...,We have studied two particle wave functions pa...,This article investigates the bone characteris...,The bosoniccharacter of the two-particle wave ...,The authors use a theoretical framework based ...
3,4,Polymer Quantum Mechanics and its Continuum Limit,A rather non-standard quantum representation o...,Polymer quantum mechanics emerges as a fascina...,ChatGPT,arxiv,Polymer quantum mechanics is a framework that ...,"Abstract:In this article, we explore the fasci...","The polymer representation, a distinct quantum...",Polymer quantum mechanics presents an intrigui...,...,A rather non-standard quantum representaiton o...,Polymer quantum mechanics emreges as a fascina...,A rather non-standard quantum representation o...,Polymer quantum mechanics emerges as a fascina...,A rather non-standard quantum representation o...,Polymer quantum mechanics e merges as a fascin...,The quantum quantum quantum quantum quantum qu...,Polymerization quantum mechanics is a fascinat...,Thisapproach has been followed in a symmetric ...,This article delves into the study of polymer ...
4,5,Numerical solution of shock and ramp compressi...,A general formulation was developed to repr

In [5]:
if DEBUG:
    df = df.iloc[100:102]
print(len(df))
path_jsonl_dataframe = generate_jsonl_file(df)
path_jsonl_dataframe

2it [00:00, 80.37it/s]


'/mnt/hdd-baracuda/pdingfelder/Masterarbeit/src/..//results//arxiv_gpt-4.1-2025-04-14_batch_processing.jsonl'

# 1. Data Generation

## 1.1 Batch generation

In [6]:
domain: str = "arxiv"
model: str = "gpt-4.1-2025-04-14"
# input_path = f"../results/{domain}_{model}_batch_processing.jsonl"
input_path: str = path_jsonl_dataframe

batch_input_file = client.files.create(
    file=open(input_path, "rb"),
    purpose="batch"
)

print(batch_input_file)

FileObject(id='file-F6XoTHWtELenNxrtCXjHyQ', bytes=1574, created_at=1753013394, filename='arxiv_gpt-4.1-2025-04-14_batch_processing.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)


In [7]:
batch_input_file_id = batch_input_file.id
batch_object = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/responses",
    completion_window="24h",
    metadata={
        "description": "Arxiv data 100-102 for batch processing"
    }
)

In [8]:
batch = client.batches.retrieve(batch_object.id)
print(batch)

Batch(id='batch_687cdc92fbb8819087a0756f000cd27c', completion_window='24h', created_at=1753013394, endpoint='/v1/responses', input_file_id='file-F6XoTHWtELenNxrtCXjHyQ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1753099794, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'Arxiv data 100-102 for batch processing'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [9]:
while(batch.status != "completed"):
    time.sleep(20)
    batch = client.batches.retrieve(batch_object.id)
file_response = client.files.content(batch.output_file_id)
file_response

In [10]:
# Step 1: Replace JSON-style null/true/false with Python equivalents
safe_data = (
    file_response.text
    .replace('null', 'None')
    .replace('true', 'True')
    .replace('false', 'False')
)

# Step 2: Split into lines
lines = safe_data.strip().split('\n')

# Step 3: Parse each line using ast.literal_eval
outputs = []
for line in lines:
    obj = ast.literal_eval(line)
    output_list = obj["response"]["body"]["output"]
    for item in output_list:
        outputs.append({
            "custom_id": obj["custom_id"],
            "message_id": item["id"],
            "text": item["content"][0]["text"]
        })

# Step 4: Create DataFrame
df_outputs = pd.DataFrame(outputs)
df_outputs.head()

,custom_id,message_id,text
0,arxiv_101,msg_687cdc97a2a881909b38b6e562a4338a0485d76066...,"Stringy instantons, arising from D-brane confi..."
1,arxiv_102,msg_687cdcb4f8088194a0df9dee886713de08e6e610cb...,We present simultaneous near-infrared (NIR) an...


In [11]:
_path = f"{RESULT_DIR}/data_generation/gpt-4.1-arxiv_generations_test.parquet" if DEBUG else f"{RESULT_DIR}/data_generation/gpt-4.1-arxiv_generations.parquet"
df_outputs.to_parquet(_path)

## 1.2 Default generation without batch processing

In [12]:
# Prepare new column
model_name = "gpt-4.1"
new_col = f"direct_prompt_{model_name}"
df[new_col] = None

_path = f"{RESULT_DIR}/data_generation/gpt-4.1-arxiv_generations_no_batch_processing_test.parquet" if DEBUG else f"{RESULT_DIR}/data_generation/gpt-4.1-arxiv_generations_no_batch_processing.parquet"

# Process only first 2 rows if DEBUG mode is true
for idx, row in tqdm(df.iterrows()):
    sentences_num = len(nltk.sent_tokenize(row["abstract"]))
    title = row["title"]
    out = generate_openai_response(sentences_num, title)
    df.at[idx, new_col] = out

# ▪️ Save to Parquet
df.to_parquet(_path, index=False)
print("✅ Saved to output_with_prompts.parquet")

2it [00:09,  4.98s/it]

✅ Saved to output_with_prompts.parquet


In [13]:
df["direct_prompt_gpt-4.1"].head(10)

100    Stringy instantons—non-perturbative effects ar...
101    We present multiband photometric observations ...
Name: direct_prompt_gpt-4.1, dtype: object